In [1]:
from sklearn.decomposition import PCA
from model.TransE import *
from model.ConvKB import ConvKB
from model.utils import *
from model.CustomTripletMarginLoss import CustomTripletMarginLoss
from argparse import Namespace
import torch
import torch.nn as nn
import sys
from torch.autograd import Variable
import torch.optim as optim
from numpy import linalg as LA
import numpy as np
from math import log10,floor
import json
import pickle

In [2]:
if torch.cuda.is_available():
    net = torch.load("./data/GENE/ConvKB.pkl")
else:
    net = torch.load("./data/GENE/ConvKB.pkl", map_location=lambda storage, loc: storage)
net = list(net.items())
# 1: entity
# 2: relation
data_train = net[0][1].cpu().numpy()

In [3]:
with open("./data/GENE/db.json", "rb") as f:
    data = json.load(f)
    f.close()
with open("./data/GENE/id_dict", "rb") as f:
    id_dict = pickle.load(f)
    f.close()
processed_entity_2_id = load_data("./data/GENE/entity2id.txt", ignore_first=True)
relation_2_id = load_data("./data/GENE/relation2id.txt", ignore_first=True)
processed_id_2_entity = dict()
i = 0
with open("./data/GENE/entity2id.txt", 'r') as f:
    for line in f:
        if True and i == 0:
            i += 1
            continue
        line = line.strip()
        if line == '':
            continue
        parts = line.split("\t")
        processed_id_2_entity[int(parts[1])] = parts[0]

In [7]:
with open("./data/GENE/kNN.pkl", "rb") as f:
    nbrs = pickle.load(f)
    f.close()
distances, indices = nbrs.kneighbors(data_train)
with open("./data/GENE/indices.pkl", "wb") as f:
    pickle.dump(indices,f)
    f.close()
with open("./data/GENE/distance.pkl", "wb") as f:
    pickle.dump(distances,f)
    f.close()

In [8]:
with open("./data/GENE/kNN.pkl", "rb") as f:
    nbrs = pickle.load(f)
    f.close()
with open("./data/GENE/indices.pkl", "rb") as f:
    indices = pickle.load(f)
    f.close()
with open("./data/GENE/distance.pkl", "rb") as f:
    distances = pickle.load(f)
    f.close()

In [9]:
indices

array([[    0, 15808,  7976, ..., 14452, 22537,  7863],
       [    1,  7718,     4, ...,   613,  5057, 14658],
       [    2,     5,     4, ...,  3919,  8866,  4872],
       ...,
       [26983,  7209,  8259, ...,  6856, 24886,  9848],
       [26984,  4511, 22086, ..., 18766,  4439, 11215],
       [26985, 23972, 23673, ..., 24200, 23696, 26895]])

array([[0.        , 0.04193108, 0.1076816 , ..., 0.5199063 , 0.52899642,
        0.53109388],
       [0.        , 0.81078548, 0.82713589, ..., 1.004464  , 1.01056503,
        1.01209699],
       [0.        , 0.46836858, 0.47319053, ..., 0.71949062, 0.72112203,
        0.72738941],
       ...,
       [0.        , 0.06511898, 0.06646741, ..., 0.07155802, 0.07442586,
        0.07566636],
       [0.        , 0.165549  , 0.25395233, ..., 0.3343984 , 0.3344013 ,
        0.33586462],
       [0.        , 0.00161525, 0.00169918, ..., 0.00222375, 0.00222896,
        0.00225883]])